In [160]:
import pandas as pd
import numpy as np
import re
import json
from datetime import datetime

In [2]:
pd.set_option('display.max_columns', None)

In [80]:
with open('packets.json', 'r') as file:
    packets = json.load(file)

In [194]:
df = pd.read_json('packets.json')

In [195]:
len(df)

49569

In [197]:
def make_remove_prefix(prefix):
    def remove_prefix(s):
        if s.startswith(prefix):
            return s[1:]
        return s
    return remove_prefix

In [198]:
df['source_address'] = df['source_address'].map(make_remove_prefix('/'))
df['destination_address'] = df['destination_address'].map(make_remove_prefix('/'))

In [199]:
def make_get_flag(pos):
    def get_flag(s):
        return [i == 'true' for i in s.split('=')[1].strip('() ').split(', ')][pos]
    return get_flag

In [200]:
df['flags_reserved'] = df['flags'].map(make_get_flag(0))
df['flags_dont_fragment'] = df['flags'].map(make_get_flag(1))
df['flags_more_fragment'] = df['flags'].map(make_get_flag(2))

In [201]:
df = df.drop('flags', axis=1)

In [202]:
option_re = re.compile(r'\[Kind: (\d+) .*')
option_bytes_re = re.compile(r'.* \[Length: (\d+) bytes.*')

In [203]:
def match_re(r, default=None):
    def find(s):
        def match_or_default():
            l = r.findall(s)
            if l:
                return l[0]
            return default
        
        try:
            if np.isnan(s):
                return default
            else:
                return match_or_default()
        except:
            return match_or_default()

    return find

In [233]:
df['window'] = df['window'].map(int)
df['ttl'] = df['ttl'].map(int)
df['identification'] = df['identification'].map(int)
df['sequence_number'] = df['sequence_number'].map(int)
df['acknowledgment_number'] = df['acknowledgment_number'].map(int)
df['reserved'] = df['reserved'].map(int)

In [205]:
for i in range(1, 7):
    df[f'option{i}_length'] = df[f'option{i}'].map(match_re(option_bytes_re, default=0))
    df[f'option{i}_length'] = df[f'option{i}_length'].map(int)
    df[f'option{i}'] = df[f'option{i}'].map(match_re(option_re, default=np.nan))

In [206]:
df = df.drop(['protocol', 'urg', 'version', 'ihl', 'fragment_offset', 'destination_port', 'tos',
              'urgent_pointer', 'type'], axis=1)
# every row has: protocol = '6 (TCP)'; urg = 'false'; version = '4 (IPv4)'; ihl = '5 (20 [bytes])';
# fragment_offset = '0 (0 [bytes])'; destination_port = '8070 (unknown)';
# tos = '[precedence: 0 (Routine)] [tos: 0 (Default)] [mbz: 0]'; urgent_pointer = '0';
# type = '0x0800 (IPv4)'

In [236]:
df['rst'] = df['rst'].map(lambda d: d == 'true')
df['psh'] = df['psh'].map(lambda d: d == 'true')
df['fin'] = df['fin'].map(lambda d: d == 'true')
df['syn'] = df['syn'].map(lambda d: d == 'true')
df['ack'] = df['ack'].map(lambda d: d == 'true')

In [209]:
data_offset_re = re.compile(r'^\d+ \((\d+) \[bytes.*')

In [210]:
df['data_offset'] = df['data_offset'].map(match_re(data_offset_re, 0))

In [211]:
df['captured_at'] = df['captured_at'].map(lambda d: datetime.fromisoformat(d.split('Z')[0]))

In [223]:
length_re = re.compile(r'^(\d+) \[?bytes\]?')

In [227]:
df['total_length'] = df['total_length'].map(match_re(length_re, 0))
df['original_length'] = df['original_length'].map(match_re(length_re, 0))

In [239]:
port_re = re.compile(r'^(\d+) \(unknown\)')

In [243]:
df['source_port'] = df['source_port'].map(match_re(port_re))

In [247]:
df

,source_address,header_checksum,destination_address,option3,window,option4,option1,option2,checksum,ttl,rst,identification,data_offset,source_addressmac,captured_at,total_length,sequence_number,psh,acknowledgment_number,fin,original_length,syn,reserved,ack,source_port,destination_addressmac,hex_stream,zzz,option5,option6,flags_reserved,flags_dont_fragment,flags_more_fragment,option1_length,option2_length,option3_length,option4_length,option5_length,option6_length
0,170.231.187.126,0x4bd8,10.128.0.2,1,2144,4,2,1,0xd0b7,238,False,4104,28,42:01:0a:80:00:01,2020-11-15 11:08:56.401955,48,18896120,False,0,False,62,True,0,False,50973,42:01:0a:80:00:02,NaN,NaN,NaN,NaN,False,False,False,4,0,0,2,0,0
1,170.231.187.126,0x4bdf,10.128.0.2,NaN,2144,NaN,NaN,NaN,0x90ae,238,False,4105,20,42:01:0a:80:00:01,2020-11-15 11:08:56.550260,40,18896121,False,3366166653,False,54,False,0,True,50973,42:01:0a:80:00:02,NaN,NaN,NaN,NaN,False,False,False,0,0,0,0,0,0
2,170.231.187.126,0x4afa,10.128.0.2,NaN,2144,NaN,NaN,NaN,0xc415,238,False,4106,20,42:01:0a:80:00:01,2020-11-15 11:08:56.557755,268,18896121,True,3366166653,False,282,False,0,True,50973,42:01:0a:80:00:02,50 4f 53 54 20 2f 61 70 69 2f 76 31 2f 61 75 7...,POST /api/v1/auth/device/signin HTTP/1.1\r\nHo...,NaN,NaN,False,False,False,0,0,0,0,0,0
3,170.231.187.126,0x4b8d,10.128.0.2,NaN,2144,NaN,NaN,NaN,0x9463,238,False,4107,20,42:01:0a:80:00:01,2020-11-15 11:08:56.557921,120,18896349,True,3366166653,False,134,False,0,True,50973,42:01:0a:80:00:02,7b 22 75 73 65 72 6e 61 6d 65 22 3a 22 33 38 3...,"{""username"":""3841bf2d-6481-406a-9f41-6dba77cda...",NaN,NaN,False,False,False,0,0,0,0,0,0
4,170.231.187.126,0x4bdc,10.128.0.2,NaN,1889,NaN,NaN,NaN,0x8f79,238,False,4108,20,42:01:0a:80:00:01,2020-11-15 11:08:56.803207,40,18896429,False,3366166908,True,54,False,0,True,50973,42:01:0a:80:00:02,NaN,NaN,NaN,NaN,False,False,False,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49564,138.94.53.1,0xfa73,10.128.0.2,NaN,2144,NaN,NaN,NaN,0xa906,238,False,2171,20,42:01:0a:80:00:01,2020-11-22 19:09:39.116731,40,3398835,False,2911016762,False,54,False,0,True,62306,42:01:0a:80:00:02,NaN,NaN,NaN,NaN,False,False,False,0,0,0,0,0,0
49565,138.94.53.1,0xf8c0,10.128.0.2,NaN,2144,NaN,NaN,NaN,0xbf54,238,False,2172,20,42:01:0a:80:00:01,2020-11-22 19:09:39.123390,474,3398835,True,2911016762,False,488,False,0,True,62306,42:01:0a:80:00:02,50 4f 53 54 20 2f 61 70 69 2f 76 31 2f 6d 65 7...,POST /api/v1/message HTTP/1.1\r\nHost: 34.68.1...,NaN,NaN,False,False,False,0,0,0,0,0,0
49566,138.94.53.1,0xfa5c,10.128.0.2,NaN,2144,NaN,NaN,NaN,0x8c2e,238,False,2173,20,42:01:0a:80:00:01,2020-11-22 19:09:39.123430,61,3399269,True,2911016762,False,75,False,0,True,62306,42:01:0a:80:00:02,7b 22 74 65 6d 70 65 72 61 74 75 72 61 22 3a 3...,"{""temperatura"":23.64}",NaN,NaN,False,False,False,0,0,0,0,0,0
49567,138.94.53.1,0xfa70,10.128.0.2,NaN,1889,NaN,NaN,NaN,0xa73e,238,False,2174,20,42:01:0a:80:00:01,2020-11-22 19:09:39.319831,40,3399290,False,2911017017,True,54,False,0,True,62306,42:01:0a:80:00:02,NaN,NaN,NaN,NaN,False,False,False,0,0,0,0,0,0


In [255]:
df = df.set_index('captured_at').sort_index()

In [256]:
df

,source_address,header_checksum,destination_address,option3,window,option4,option1,option2,checksum,ttl,rst,identification,data_offset,source_addressmac,total_length,sequence_number,psh,acknowledgment_number,fin,original_length,syn,reserved,ack,source_port,destination_addressmac,hex_stream,zzz,option5,option6,flags_reserved,flags_dont_fragment,flags_more_fragment,option1_length,option2_length,option3_length,option4_length,option5_length,option6_length
captured_at,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-11-15 11:08:56.401955,170.231.187.126,0x4bd8,10.128.0.2,1,2144,4,2,1,0xd0b7,238,False,4104,28,42:01:0a:80:00:01,48,18896120,False,0,False,62,True,0,False,50973,42:01:0a:80:00:02,NaN,NaN,NaN,NaN,False,False,False,4,0,0,2,0,0
2020-11-15 11:08:56.550260,170.231.187.126,0x4bdf,10.128.0.2,NaN,2144,NaN,NaN,NaN,0x90ae,238,False,4105,20,42:01:0a:80:00:01,40,18896121,False,3366166653,False,54,False,0,True,50973,42:01:0a:80:00:02,NaN,NaN,NaN,NaN,False,False,False,0,0,0,0,0,0
2020-11-15 11:08:56.557755,170.231.187.126,0x4afa,10.128.0.2,NaN,2144,NaN,NaN,NaN,0xc415,238,False,4106,20,42:01:0a:80:00:01,268,18896121,True,3366166653,False,282,False,0,True,50973,42:01:0a:80:00:02,50 4f 53 54 20 2f 61 70 69 2f 76 31 2f 61 75 7...,POST /api/v1/auth/device/signin HTTP/1.1\r\nHo...,NaN,NaN,False,False,False,0,0,0,0,0,0
2020-11-15 11:08:56.557921,170.231.187.126,0x4b8d,10.128.0.2,NaN,2144,NaN,NaN,NaN,0x9463,238,False,4107,20,42:01:0a:80:00:01,120,18896349,True,3366166653,False,134,False,0,True,50973,42:01:0a:80:00:02,7b 22 75 73 65 72 6e 61 6d 65 22 3a 22 33 38 3...,"{""username"":""3841bf2d-6481-406a-9f41-6dba77cda...",NaN,NaN,False,False,False,0,0,0,0,0,0
2020-11-15 11:08:56.803207,170.231.187.126,0x4bdc,10.128.0.2,NaN,1889,NaN,NaN,NaN,0x8f79,238,False,4108,20,42:01:0a:80:00:01,40,18896429,False,3366166908,True,54,False,0,True,50973,42:01:0a:80:00:02,NaN,NaN,NaN,NaN,False,False,False,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-11-22 19:09:39.116731,138.94.53.1,0xfa73,10.128.0.2,NaN,2144,NaN,NaN,NaN,0xa906,238,False,2171,20,42:01:0a:80:00:01,40,3398835,False,2911016762,False,54,False,0,True,62306,42:01:0a:80:00:02,NaN,NaN,NaN,NaN,False,False,False,0,0,0,0,0,0
2020-11-22 19:09:39.123390,138.94.53.1,0xf8c0,10.128.0.2,NaN,2144,NaN,NaN,NaN,0xbf54,238,False,2172,20,42:01:0a:80:00:01,474,3398835,True,2911016762,False,488,False,0,True,62306,42:01:0a:80:00:02,50 4f 53 54 20 2f 61 70 69 2f 76 31 2f 6d 65 7...,POST /api/v1/message HTTP/1.1\r\nHost: 34.68.1...,NaN,NaN,False,False,False,0,0,0,0,0,0
2020-11-22 19:09:39.123430,138.94.53.1,0xfa5c,10.128.0.2,NaN,2144,NaN,NaN,NaN,0x8c2e,238,False,2173,20,42:01:0a:80:00:01,61,3399269,True,2911016762,False,75,False,0,True,62306,42:01:0a:80:00:02,7b 22 74 65 6d 70 65 72 61 74 75 72 61 22 3a 3...,"{""temperatura"":23.64}",NaN,NaN,False,False,False,0,0,0,0,0,0


In [254]:
df[~df['zzz'].isna()]

,source_address,header_checksum,destination_address,option3,window,option4,option1,option2,checksum,ttl,rst,identification,data_offset,source_addressmac,total_length,sequence_number,psh,acknowledgment_number,fin,original_length,syn,reserved,ack,source_port,destination_addressmac,hex_stream,zzz,option5,option6,flags_reserved,flags_dont_fragment,flags_more_fragment,option1_length,option2_length,option3_length,option4_length,option5_length,option6_length
captured_at,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-11-15 11:08:56.557755,170.231.187.126,0x4afa,10.128.0.2,NaN,2144,NaN,NaN,NaN,0xc415,238,False,4106,20,42:01:0a:80:00:01,268,18896121,True,3366166653,False,282,False,0,True,50973,42:01:0a:80:00:02,50 4f 53 54 20 2f 61 70 69 2f 76 31 2f 61 75 7...,POST /api/v1/auth/device/signin HTTP/1.1\r\nHo...,NaN,NaN,False,False,False,0,0,0,0,0,0
2020-11-15 11:08:56.557921,170.231.187.126,0x4b8d,10.128.0.2,NaN,2144,NaN,NaN,NaN,0x9463,238,False,4107,20,42:01:0a:80:00:01,120,18896349,True,3366166653,False,134,False,0,True,50973,42:01:0a:80:00:02,7b 22 75 73 65 72 6e 61 6d 65 22 3a 22 33 38 3...,"{""username"":""3841bf2d-6481-406a-9f41-6dba77cda...",NaN,NaN,False,False,False,0,0,0,0,0,0
2020-11-15 11:10:32.960528,170.231.187.126,0x4af2,10.128.0.2,NaN,2144,NaN,NaN,NaN,0x88e8,238,False,4114,20,42:01:0a:80:00:01,268,18981399,True,1200034468,False,282,False,0,True,50209,42:01:0a:80:00:02,50 4f 53 54 20 2f 61 70 69 2f 76 31 2f 61 75 7...,POST /api/v1/auth/device/signin HTTP/1.1\r\nHo...,NaN,NaN,False,False,False,0,0,0,0,0,0
2020-11-15 11:10:32.960651,170.231.187.126,0x4b85,10.128.0.2,NaN,2144,NaN,NaN,NaN,0x5936,238,False,4115,20,42:01:0a:80:00:01,120,18981627,True,1200034468,False,134,False,0,True,50209,42:01:0a:80:00:02,7b 22 75 73 65 72 6e 61 6d 65 22 3a 22 33 38 3...,"{""username"":""3841bf2d-6481-406a-9f41-6dba77cda...",NaN,NaN,False,False,False,0,0,0,0,0,0
2020-11-15 11:12:09.364875,170.231.187.126,0x4ae8,10.128.0.2,NaN,2144,NaN,NaN,NaN,0x4cd9,238,False,4124,20,42:01:0a:80:00:01,268,19066870,True,2455090346,False,282,False,0,True,50555,42:01:0a:80:00:02,50 4f 53 54 20 2f 61 70 69 2f 76 31 2f 61 75 7...,POST /api/v1/auth/device/signin HTTP/1.1\r\nHo...,NaN,NaN,False,False,False,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-11-22 19:06:25.088245,138.94.53.1,0xfa7c,10.128.0.2,NaN,2144,NaN,NaN,NaN,0x2d61,238,False,2141,20,42:01:0a:80:00:01,61,3297622,True,1475074318,False,75,False,0,True,62211,42:01:0a:80:00:02,7b 22 74 65 6d 70 65 72 61 74 75 72 61 22 3a 3...,"{""temperatura"":23.64}",NaN,NaN,False,False,False,0,0,0,0,0,0
2020-11-22 19:08:02.100206,138.94.53.1,0xf8cf,10.128.0.2,NaN,2144,NaN,NaN,NaN,0xdf48,238,False,2157,20,42:01:0a:80:00:01,474,3347818,True,134865517,False,488,False,0,True,62718,42:01:0a:80:00:02,50 4f 53 54 20 2f 61 70 69 2f 76 31 2f 6d 65 7...,POST /api/v1/message HTTP/1.1\r\nHost: 34.68.1...,NaN,NaN,False,False,False,0,0,0,0,0,0
2020-11-22 19:08:02.106091,138.94.53.1,0xfa6b,10.128.0.2,NaN,2144,NaN,NaN,NaN,0xac22,238,False,2158,20,42:01:0a:80:00:01,61,3348252,True,134865517,False,75,False,0,True,62718,42:01:0a:80:00:02,7b 22 74 65 6d 70 65 72 61 74 75 72 61 22 3a 3...,"{""temperatura"":23.64}",NaN,NaN,False,False,False,0,0,0,0,0,0


In [249]:
df.to_csv('tcp_data.csv')